# Autor: Jesús Ernesto Carro Martínez.

## Ejercicio 1

En clase vimos que la función `bits` permite ver la representación binaria de números de punto flotante (`Float64`) o de enteros (e.g., `Int64`). En particular, para números de punto flotante, el primer bit es el signo, los siguiente 11 son el *exponente* y el resto es la representación binaria (sin el 1 del principio), del número en cuestión.

La pregunta es ¿qué significa el exponente?

```julia
bits(0.1)
"0111111111110000000000000000000000000000000000000000000000000000"
```

### Respuesta: 


En el caso de los lenguajes de programación que no hacen [escala fija de números enteros](https://en.wikipedia.org/wiki/Scale_factor_(computer_science), como son Pascal, C y este nuevo lenguaje que estamos aprendiendo, Julia; la forma más fácil de manejar números reales dentro de una computadora es con números de punto flotante. Usando esta técnica un número está representado en bits por tres partes: signo, exponente, y la mantista. Ésto es similar a la notación científica, la cual se utiliza para representar números grandes o pequeños (por ejemplo $-0.35 \times 10^{8}$). El signo es negativo, el exponente es $8$ y la mantista es $0.35$. Dentro de una computadora éstos se mantienen en el sistema binario y son puestos en campos separados para números de punto flotante.

Los números pueden ser presentados tanto en 64-bits o 32-bits, dependiendo de las características de la computadora (este trabajo se desarrolla en base a la notación de 64-bits); además, los números son expresados siguiendo el [estándar del IEEE para aritmética en punto flotante](https://en.wikipedia.org/wiki/IEEE_floating_point).

La forma en que son representados los números flotantes en bits bajo el formato de 64-bits y siguiendo el estándar del IEEE siguen el siguiente orden:

1) 1-bit para el signo.
2) 11-bits para el exponente.
3) 52-bits para el número central, *significand*.

El valor de los números expresados de esta forma es:

$$(-1)^{signo} \times 2^{exponente} \times (0.significand+1.0)\cdot\cdot\cdot\cdot\cdot\cdot\cdot\cdot\cdot\cdot\cdot\cdot(1)$$

En el formato de 64-bits, el exponente se mantiene en notación de exceso de $1023$, mientras el *significand* son números fraccionales expresados en forma similar que los enteros, pero en este caso cada entrada viene determinada por la potencia $2^{-n}$, siendo $n$ el número de entrada. Por ejemplo, si desearamos expresar los números $0.625$ y $0.1$ en forma binaria se tendría que escribir como:

$$0.625_{10} = 0.101_{2} = 1\cdot 2^{-1} + 0\cdot 2^{-2} + 1\cdot 2^{-3}$$

$$0.1_{10} = 0.001100110011001100110011001100110011001100110011010_{2} = 0\cdot 2^{-1} + 0\cdot 2^{-2} + 1\cdot 2^{-3} + 1\cdot 2^{-4} + 0\cdot 2^{-5} + ... $$

En los ejemplos anteriores los subíndices hacen referencia a la base que se está empleando. Ahora, utilicemos ésto en un ejemplo en concreto:


In [2]:
bits(12.0)

"0100000000101000000000000000000000000000000000000000000000000000"

El primer dígito ($0$) representa el signo del número (positivo), los siguientes once ($10000000010$) corresponden al exponente y 100... al *significand*.

El exponente $10000000010$ será igual a $1026$ en la base decimal, pero al emplear una notación de exceso de $1023$ el exponente será igual a $1026-1023=3$, mientras $0.100..._{2} = 0.5_{10}$; de tal forma, empleando la ecuación (1):

$$ (1) \times 2^{3} \times 1.5 = 12.0$$

Justo el resultado esperado. Por último, demos un ejemplo más, en el cual obtendremos exponentes negativos!!! Será muy sencillo pero igualmente ilustrativo, se tiene:

In [10]:
bits(0.625)

"0011111111100100000000000000000000000000000000000000000000000000"

De forma análoga al caso anterior, el primer dígito ($0$) representa el signo del número (positivo), los siguientes once ($01111111110$) corresponden al exponente y 0100... al *significand*.

El exponente $01111111110$ será igual a $1022$, pero al emplear una notación de exceso de $1023$ el exponente será igual a $1022-1023=-1$, mientras $0.0100..._{2} = 0.25_{10}$; de tal forma, empleando la ecuación (1):

$$(1) \times 2^{-1} \times 1.25 = 0.625$$

Justo el resultado que esperábamos encontrar.

De igaul forma que en el ejemplo anterior, lo antes escrito se confirma. **En conclusión, el exponente es el exponente del 2 en la operación:**

$$(-1)^{signo} \times 2^{exponente} \times (0.significand+1.0)$$

**para expresar los números flotantes en forma binaria.**

## Ejercicio 2

- ¿Cuál es el número más chico que sumado (o restado) a 1.0 me da algo distinto de 1.0?
(Este número se bautiza como el "épsilon de la máquina")

- ¿Cuál es el número más chico que sumado (o restado) a 10.0 me da algo distinto de 10.0?


### Respuesta:

Una forma sencilla de contestar esta pregunta sería emplear la función `eps()` con la cual cuenta `Julia`; sin embargo, también podemos recurrir a un while para encontrar la épsilon de la máquina, ésto es:

In [14]:
macheps1 = 1.0;

while 1.0 + (macheps1/2.0) > 1.0
    macheps1 = macheps1/2.0
end

In [15]:
macheps10 = 1.0;

while 10.0 + (macheps10/2.0) > 10.0
    macheps10 = macheps10/2.0
end

Es importante comprender lo que se está haciendo en los bucles anteriores, lo cual está fuertemente relacionado con el Ejercicio 1, pues el épsilon de la maquina es el límite superior del error relativo debido a la aritmética de punto flotante. Es decir, éste debería ser la diferencia máxima esperada entre un verdadero número de punto flotante y otro que es calculado en la computadora debido al número finito de bits usados para almacenar un número de punto flotante y, de hecho, ésto implica que los épsilon de máquina de 1.0 y 10.0 deberán de ser distintos.

Ahora bien, pasemos a describir el código: en el bucle anterior, la división por 2 representa mover en un bit hacia la derecha en cada iteración; es decir, ir saltando entre los 52 bits del *significand*. El bucle inicia en 1.0 o $2^{0}$ pero en la primer iteración se divide sobre 2 y se comprueba si $2^{-1}$ (ésto significa mover en un bit) es el épsilon de la máquina, para lo cual se suma al valor estudiado en cuestión y si cumple la condición de ser aún mayor que dicho dato entonces se realiza otra iteración con $2^{-2}$ (se mueve en un bit).

Lo anterior se repetirá hasta que 1.0 + macheps1 sea igual a 1.0, ésto implicará que los bits del *significand* ya no son suficientes para expresar el número 1.0 + macheps1; es decir, hemos superado los 52 bits. Además, ésto implicará épsilon de la máquina diferentes para 1.0 y 10.0.

**En base a lo anterior, podemos asegurar que los épsilon de la máquina son:**

In [10]:
macheps1

2.220446049250313e-16

In [11]:
macheps10

1.7763568394002505e-15

**para 1.0 y 10.0, respectivamente**. De hecho, se puede observar:

In [9]:
1.0 + macheps1

1.0000000000000002

In [10]:
10.0 + macheps10

10.000000000000002

Como nos solicitaba el Ejercicio y, justo como se mencionó previamente, los épsilon de la máquina son diferentes para 1.0 y 10.0. Podemos comparar con el épsilon de la máquina dado por la función `eps()`:

In [15]:
abs(macheps1-eps(1.0))

0.0

In [13]:
abs(macheps10-eps(10.0))

0.0

Son exactamente iguales! (O bueno, hasta donde la computadora alcanza a "ver", lo son :) ). En conclusión, los resultados obtenidos son una buena aproximación a la épsilon de la máquina y cumplen con lo deseado en el ejercicio; además, confirman lo descrito previamente.

Para no generar dudas, veamos que ocurre con la resta, es de esperarse que sean iguales; sin embargo, siempre es bueno dejar todo claro:

In [4]:
macheps1 = 1.0;

while 1.0 - (macheps1/2.0) < 1.0
    macheps1 = macheps1/2.0
end

In [8]:
macheps10 = 1.0;

while 10.0 - (macheps10/2.0) < 10.0
    macheps10 = macheps10/2.0
end

In [9]:
macheps1

1.1102230246251565e-16

In [10]:
macheps10

1.7763568394002505e-15

Sorpresa!! Pues, para 1.0 el resultado es diferente en la suma; de hecho, para la resta el épsilon de la máquina es la mitad del épsilon de la suma, lo cual no ocurre con 10.0. Para estar seguros:

In [11]:
1.0 + macheps1

1.0

In [12]:
1.0 - macheps1

0.9999999999999999

**Por tanto, si sumamos el nuevo valor de `macheps1` no se obtiene ningún cambio para 1.0, entonces los épsilon obtenidos para ambas operaciones son correctos, aunque no coincidan.**

Sería interesante discutir estos resultados en términos de bits; sin embargo, lo obtenido hasta ahora es suficiente para el Ejercicio, si el lector está interesado en más información le recomiendo los siguientes links: [1](https://en.wikipedia.org/wiki/Double-precision_floating-point_format) y [2](http://stackoverflow.com/questions/27511349/matlab-calculating-machine-epsilon).

## Ejercicio 3

Define una functión llamada `derivada1` que implemente el cálculo de:

\begin{equation}
\frac{\Delta f(x)}{\Delta x}(x_0) = \frac{f(x_0+h)-f(x_0)}{h}
\end{equation}

Obviamente esta función es una aproximación *finita* de la derivada. La pregunta es ¿qué tan buena es esta aproximación, pensando en términos de $h$?

- La primer sub-pregunta es ¿cómo se responde este tipo de pregunta?
- Uno tiene que tomar un caso concreto (para $f(x_0,h)$); sugiero empezar con un polinomio con coeficientes enteros, y después hacer el mismo tipo de pruebas con una función trascendental.
- ¿Hay algo *sorprendente* en el comportamiento de `derivada1` como función de $h$? ¿A qué lo puedes atribuir?
- ¿Puedes hacer que la función `derivada1` funcione para funciones más o menos arbitrarias, que el usuario define?
- ¿Puedes explicar de alguna manera los resultados que has obtenido?

### Respuesta:

Definimos la función `derivada1` en base a la ecuación que aparece en el enunciado del Ejercicio:

In [20]:
function derivada1(f,x,h)
    return ((f(x+h)-f(x))/h)
end

derivada1 (generic function with 1 method)

En la función `derivada1`, *f* hace referencia a la función de la cual se desea conocer su derivada en algún punto $x_{0}$ (el cual es representado por *x* dentro de `derivada1`) y se puede introducir manualmente por el usuario.

Por último, la función `derivada1` también permite cambiar el valor de *h*, lo cual será una gran ventaja para analizar el comportamiento de la derivada analítica con respecto a *h*. Ya definida nuestra función y habiendo explicada sus características, pasemos a contestar las preguntas que se nos plantean en el ejercicio.

**Considero que la forma más rápida de contestar a la pregunta de qué tan buena es la aproximación definida por la función `derivada1` es comparar con las derivadas de algunas funciones conocidas.** Continuando con esta idea y siguiendo la sugerencia del segundo sub-inciso tomaré la función polinomial:

In [22]:
f1(x) = x + x^2 + x^3 + x^4 + x^5 

f1 (generic function with 1 method)

La derivada de la función anterior es fácil de encontrar:

In [23]:
df1(x) = 1 + 2*x + 3*(x^2) + 4*(x^3) + 5*(x^4)

df1 (generic function with 1 method)

Es interesante estudiar como cambia la función `derivada1` conforme *h* se hace más pequeña, para ello recurriremos a un *if*, en el cual se reducirá el valor de *h* en cada paso, ésto permitirá ver la convergencia al resultado real para diferentes valores de *h*. Vamos:

In [35]:
for i = 1:30
    h = 1.0 / (10.0^i)
    println(h, "    ",abs(df1(0.5)-derivada1(f1,0.5,h)))
end

0.1    0.5835999999999988
0.01    0.05305351000000247
0.001    0.005255503500857905
0.0001    0.0005250550033881751
1.0e-5    5.250052450689324e-5
1.0e-6    5.250020997493721e-6
1.0e-7    5.234685147570417e-7
1.0e-8    7.271796675922815e-8
1.0e-9    1.4932663816580316e-7
1.0e-10    2.9476257168425946e-7
1.0e-11    5.845877694810042e-6
1.0e-12    0.00012738088526020874
1.0e-13    0.0013159090467524948
1.0e-14    0.0013159090467524948
1.0e-15    0.10123598126301614
1.0e-16    0.8783920985006262
1.0e-17    3.5625
1.0e-18    3.5625
1.0e-19    3.5625
1.0e-20    3.5625
1.0e-21    3.5625
1.0e-22    3.5625
1.0000000000000001e-23    3.5625
1.0000000000000001e-24    3.5625
9.999999999999999e-26    3.5625
9.999999999999999e-27    3.5625
1.0e-27    3.5625
1.0000000000000001e-28    3.5625
1.0000000000000001e-29    3.5625
9.999999999999999e-31    3.5625


Al principio de la lista podemos ver que conforme se reduce *h*, se nota una mejor aproximación al valor real de la derivada de f1 en $0.5$, justo como era de esperarse; sin embargo, también se aprecian algunos casos extraños, pues al reducir aún más el valor de *h* llega un punto en el cual la diferencia crece y, posteriormente, se hace constante. Veamos que sucede en otro ejemplo:

In [25]:
f2(x) = x^(1/2)

f2 (generic function with 1 method)

La derivada es sencilla de calcular, se tendrá:

In [26]:
df2(x) = (1/2)*(x^(-1/2))

df2 (generic function with 1 method)

Obtengamos la misma lista que en el caso anterior y evaluando en 0.5:

In [37]:
for i = 1:30
    h = 1.0 / (10.0^i)
    println(h, "    ",abs(df2(0.5)-derivada1(f2,0.5,h)))
end

0.1    0.03220790063718926
0.01    0.003500614412807246
0.001    0.0003532002785547883
0.0001    3.535180434754981e-5
1.0e-5    3.5355080834076347e-6
1.0e-6    3.5355338356968247e-7
1.0e-7    3.5807553921962665e-8
1.0e-8    8.051978306333751e-9
1.0e-9    7.466535978384314e-8
1.0e-10    1.856876622463588e-7
1.0e-11    5.7368027853721415e-6
1.0e-12    0.0001167591052478878
1.0e-13    0.00010528549967714351
1.0e-14    0.007666275672698952
1.0e-15    0.04097296641145365
1.0e-16    0.7071067811865476
1.0e-17    0.7071067811865476
1.0e-18    0.7071067811865476
1.0e-19    0.7071067811865476
1.0e-20    0.7071067811865476
1.0e-21    0.7071067811865476
1.0e-22    0.7071067811865476
1.0000000000000001e-23    0.7071067811865476
1.0000000000000001e-24    0.7071067811865476
9.999999999999999e-26    0.7071067811865476
9.999999999999999e-27    0.7071067811865476
1.0e-27    0.7071067811865476
1.0000000000000001e-28    0.7071067811865476
1.0000000000000001e-29    0.7071067811865476
9.999999999999999e-31

Al igual que con el polinomio, la aproximación por medio de la función `derivada1` tiende a un número constante para *h* lo suficientemente pequeña. De hecho, en ambos casos el mínimo en la diferencia entre el dato real y el aproximado se alcanza para la misma *h*, es decir, en $1.08\times 10^{-8}$; sin embargo, después de dicho valor la diferencia se hace mayor hasta asumir un valor constante.

Por tanto, **sí existe un comportamiento inesperado (sorprendente) al estudiar a `derivada1` como función de *h* **, pues personalmente hubiese esperado que la diferencia tendiera a cero conforme *h* se hace más pequeña.

Se nos pregunta a qué podemos atribuir el comportamiento inesperado, **personalmente considero que al hacer *h* muy pequeña la diferencia dada por:**

$$ f(x_0+h)-f(x_0) $$

**se hará demasiado chica, de tal forma que la computadora incurrirá en un error de redondeo que terminará por hacer constante el valor de toda la derivada.**

**Por otro lado, existe una región donde el error disminuye considerablemente, es decir entre $1.0$ y $1\times 10^{-8}$, lo cual se debe a que $ f(x_0+h)-f(x_0) $ todavía no es tan pequeño como para ser redondeado por la máquina, representando una mejor convergencia conforme `h` se hace más pequeña.**

Lo siguiente es comprobar si la función es útil para cualquier función arbitraria, la mejor forma de comprobar ésto es con la práctica, definimos la función:

In [17]:
f3(x) = 30.0*x^2 + (1/2)*x^(-1/3) + (1/111)*x^(π)

f3 (generic function with 1 method)

Su derivada es:

In [18]:
df3(x) = 60.0*x - (1/6)*x^(-4/3) + (π/111)*x^(π-1.0)

df3 (generic function with 1 method)

Comparamos:

In [21]:
for i = 1:30
    h = 1.0 / (10.0^i)
    println(h, "    ",abs(df3(0.5)-derivada1(f3,0.5,h)))
end

0.1    3.050002864501902
0.01    0.3056523829532871
0.001    0.030572842383531196
0.0001    0.0030573615190938597
1.0e-5    0.00030573663558897124
1.0e-6    3.0570789927963915e-5
1.0e-7    3.024824419384231e-6
1.0e-8    3.2476202349585037e-7
1.0e-9    4.1161300785574895e-6
1.0e-10    9.445200593205527e-6
1.0e-11    0.00015155374774167285
1.0e-12    0.0029937246907856263
1.0e-13    0.010099152048386628
1.0e-14    0.09891699401839915
1.0e-15    1.164731097658553
1.0e-16    5.940696259942452
1.0e-17    29.586440528062557
1.0e-18    29.586440528062557
1.0e-19    29.586440528062557
1.0e-20    29.586440528062557
1.0e-21    29.586440528062557
1.0e-22    29.586440528062557
1.0000000000000001e-23    29.586440528062557
1.0000000000000001e-24    29.586440528062557
9.999999999999999e-26    29.586440528062557
9.999999999999999e-27    29.586440528062557
1.0e-27    29.586440528062557
1.0000000000000001e-28    29.586440528062557
1.0000000000000001e-29    29.586440528062557
9.999999999999999e-31    29.

**De igual forma que en los casos anteriores, la aproximación es correcta hasta cierto valor de `h` $(1.08\times 10^{-8})$, pero después se presenta el mismo comportamiento inesperado. Por tanto, la función `derivada1` funciona correctamente para funciones más o menos arbitrarias, pero presenta el comportamiento inesperado ya mencionado.**

Los resultados que necesitarían explicación son aquellos que se presentan cuando `h` se hace muy pequeña, pero esos ya fueron justificados previamente; por tanto, podemos dar por terminado este Ejercicio.

## Ejercicio 4

Este ejercicio es equivalente al anterior, pero considerando la función `derivada2` que implemente el cálculo de:

$$
\frac{\Delta f(x)}{\Delta x}(x_0) = \frac{f(x_0+h)-f(x_0-h)}{2h}
$$

¿Puedes explicar por qué la convergencia es mejor en este caso?


### Respuesta:

Definimos la función `derivada2` en base a la ecuación que aparece en el enunciado:

In [28]:
function derivada2(f,x,h)
    return ((f(x+h)-f(x-h))/(2.0*h))
end

derivada2 (generic function with 1 method)

Como podrá darse cuenta el lector, la función `derivada2` tiene los mismos argumentos que la función `derivada1` y, por tanto, no es necesario describirla. 

Ahora, se nos pide comprobar que la convergencia de esta función es mejor que la del Ejercicio anterior, para lo cual hacemos los mismos ejemplos del ejercicio anterior:

In [29]:
for i = 1:30
    h = 1.0 / (10.0^i)
    println(h, "    ",abs(df1(0.5)-derivada2(f1,0.5,h)))
end

0.1    0.05509999999999904
0.01    0.000550010000001766
0.001    5.500000923142068e-6
0.0001    5.500047262074759e-8
1.0e-5    5.42367484257511e-10
1.0e-6    5.715250495086366e-11
1.0e-7    5.567528660321841e-10
1.0e-8    1.1655700404844538e-8
1.0e-9    1.720681552797032e-8
1.0e-10    2.9476257168425946e-7
1.0e-11    5.845877694810042e-6
1.0e-12    0.00012738088526020874
1.0e-13    0.00034942549018524005
1.0e-14    0.0013159090467524948
1.0e-15    0.10123598126301614
1.0e-16    0.8783920985006262
1.0e-17    3.5625
1.0e-18    3.5625
1.0e-19    3.5625
1.0e-20    3.5625
1.0e-21    3.5625
1.0e-22    3.5625
1.0000000000000001e-23    3.5625
1.0000000000000001e-24    3.5625
9.999999999999999e-26    3.5625
9.999999999999999e-27    3.5625
1.0e-27    3.5625
1.0000000000000001e-28    3.5625
1.0000000000000001e-29    3.5625
9.999999999999999e-31    3.5625


In [30]:
for i = 1:30
    h = 1.0 / (10.0^i)
    println(h, "    ",abs(df2(0.5)-derivada2(f2,0.5,h)))
end

0.1    0.003598904852490037
0.01    3.5361527703448026e-5
0.001    3.5355404459647133e-7
0.0001    3.5354194016790075e-9
1.0e-5    3.59963170382116e-11
1.0e-6    2.861377801366416e-12
1.0e-7    2.746943783549227e-10
1.0e-8    2.5008631832079686e-9
1.0e-9    1.9154208552585317e-8
1.0e-10    3.6942385006621947e-7
1.0e-11    1.856876622463588e-7
1.0e-12    6.124795401662997e-5
1.0e-13    0.00010528549967714351
1.0e-14    0.0021151605495731696
1.0e-15    0.014538184819804068
1.0e-16    0.1519952688739693
1.0e-17    0.7071067811865476
1.0e-18    0.7071067811865476
1.0e-19    0.7071067811865476
1.0e-20    0.7071067811865476
1.0e-21    0.7071067811865476
1.0e-22    0.7071067811865476
1.0000000000000001e-23    0.7071067811865476
1.0000000000000001e-24    0.7071067811865476
9.999999999999999e-26    0.7071067811865476
9.999999999999999e-27    0.7071067811865476
1.0e-27    0.7071067811865476
1.0000000000000001e-28    0.7071067811865476
1.0000000000000001e-29    0.7071067811865476
9.99999999999999

In [31]:
for i = 1:30
    h = 1.0 / (10.0^i)
    println(h, "    ",abs(df3(0.5)-derivada2(f3,0.5,h)))
end

0.1    0.008866065023518388
0.01    8.608538501420071e-5
0.001    8.60606224506455e-7
0.0001    8.606864554394633e-9
1.0e-5    1.3364598316911724e-10
1.0e-6    1.1994565340955887e-9
1.0e-7    1.2745775990197217e-8
1.0e-8    5.83084975858128e-8
1.0e-9    1.4515948194571138e-6
1.0e-10    8.318367800796977e-6
1.0e-11    2.6081936198352196e-5
1.0e-12    0.0012173678513853758
1.0e-13    0.0012173678513853758
1.0e-14    0.0787186899216259
1.0e-15    0.27655267795842775
1.0e-16    5.940696259942452
1.0e-17    29.586440528062557
1.0e-18    29.586440528062557
1.0e-19    29.586440528062557
1.0e-20    29.586440528062557
1.0e-21    29.586440528062557
1.0e-22    29.586440528062557
1.0000000000000001e-23    29.586440528062557
1.0000000000000001e-24    29.586440528062557
9.999999999999999e-26    29.586440528062557
9.999999999999999e-27    29.586440528062557
1.0e-27    29.586440528062557
1.0000000000000001e-28    29.586440528062557
1.0000000000000001e-29    29.586440528062557
9.999999999999999e-31    

De los ejemplos anteriores, podemos ver que la convergencia entre el valor real y el proporcionado por la función `derivada2` es mucho mejor en comparación con la función `derivada1` (la diferencia entre dichos datos se reduce en hasta en 3 órdenes de magnitud).

Sin embargo, al igual que ocurría con `derivada1`, la función `derivada2` presenta una región en la cual el error entre el valor real y el aproximado disminuye conforme `h` se hace más pequeña, pero llega un punto donde la convergencia se pierde hasta llegar a un valor constante. De forma análoga al Ejercicio 3, considero que ésto se debe a:

$$ f(x_0+h)-f(x_0-h) $$

Pues, cuando `h` disminuye entre $0.1$ y $1\times 10^{-5}$ esta diferencia no es redondeada por la máquina, obteniéndose errores pequeños; sin embargo, si se sigue haciendo más pequeña `h`, la diferencia también será reducida considerablemente hasta que la máquina la redondee, dando lugar a una mala convergencia.

Numéricamente, la derivada real $f'(x_{0})$ estará expresada por:

$$f'(x_{0})=\frac{f(x_{0}+h)-f(x_{0})}{h}+\frac{h}{2}f''(\epsilon_{0})$$

y

$$f'(x_{0})=\frac{f(x_{0}+h)-f(x_{0}-h)}{2h}+\frac{h^{2}}{3}f^{(3)}(\epsilon_{1})$$

donde $\epsilon_{0}$ está en $[x_{0},x_{0}+h]$ y $\epsilon_{1}$ pertenece a $[x_{0}-h,x_{0}+h]$. Por tanto, el error para la función `derivada2` será menor que el error de la `derivada1`, pues `h` es mucho menor que 1 (recordemos que $h^{2}<h$ si $0<h<1$), entonces estamos diciendo que los errores dados por la relación

$$\frac{h}{2}f''(\epsilon_{0}) > \frac{h^{2}}{3}f^{(3)}(\epsilon_{1})$$

nos llevan a asegurar que la función `derivada2` presentará una mejor convergencia en comparación con `derivada1`.

**En conclusión, lo anterior nos dice que la función `derivada2` es una aproximación de segundo orden, mientras la función `derivada1` representa una aproximación de primer orden o lineal; por lo cual, el error de la primera es menor, implicando una mejor convergencia.**

**También podemos decir que la función `derivada2` representa una mejor aproximación a la derivada real debido a que esta ecuación emplea datos de ambos lados de $x_{0}$, mientras la función `derivada1` usa datos de un sólo lado; de ahí que la primer función presente una mejor convergencia al resultado real.**

#### Referencias:

Burden, Richard L.; *Numerical Analysis*, Editorial Thomson, Octava Edición, 2005, pp 168-176.

## Ejercicio 5

Manda el notebook con tus resultados haciendo un pull-request al repositorio del curso. Para hacer esto:
- Haz un `fork` del repositorio del curso a tu propia cuenta en GitHub
- Una vez que tengas resuelto el notebook, súbelo a **tu** repositorio (con `git push`), en la carpeta "tareas/usuario_github/nombre_notebook"
- Desde tu repositorio, haz un "Pull Request". El *pull request* sólo se aceptará si es legible en el sentido más amplio de la palabra y, obviamente, si es correcto. Esto significa que debe ser *entendible*, a partir de lo que ustedes escriben, incluso para gente que no está en el curso. (Como GitHub es público, piensen fríamente que esto puede ser leido por *muuucha* gente allá afuera.) Si el trabajo es en colaboración, háganlo saber en el propio notebook (cosa que se debe notar en el historial de lo que envíen). Finalmente, como código ético, incluyan cualquier fuente de inspiración, incluso cuando se trate del notebook de otro compañero/equipo del curso.

# Autor: Jesús Ernesto Carro Martínez.